In [1]:
!pip install pymupdf pandas openai langchain-text-splitters

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.9/24.9 MB 38.3 MB/s eta 0:00:00


In [3]:
from google.colab import userdata

In [13]:
# # UPSC Newspaper Analysis Pipeline - LangChain Enhanced Version
# # Using LangChain's RecursiveCharacterTextSplitter for semantic chunking

# import fitz  # PyMuPDF
# import pandas as pd
# import json
# import time
# import re
# from google.colab import files
# from google.colab import userdata
# from openai import OpenAI
# from langchain_text_splitters import RecursiveCharacterTextSplitter

# # --- CONFIGURATION ---
# OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
# CHUNK_SIZE = 1000  # Characters (LangChain works with characters, not words)
# CHUNK_OVERLAP = 100  # Characters - overlap for context continuity
# MODEL = "gpt-4o"

# def clean_text(text):
#     """Clean and normalize extracted text."""
#     # Remove excessive whitespace
#     text = re.sub(r'\s+', ' ', text)
#     # Remove page numbers and headers/footers patterns
#     text = re.sub(r'\b\d+\s*\|\s*\w+\b', '', text)
#     # Remove multiple dots
#     text = re.sub(r'\.{3,}', '...', text)
#     return text.strip()

# def extract_text_from_pdfs(uploaded_files):
#     """Enhanced PDF text extraction with cleaning."""
#     all_text = []

#     for filename, content in uploaded_files.items():
#         print(f"\n{'='*60}")
#         print(f"📄 Processing: {filename}")
#         print(f"{'='*60}")

#         try:
#             doc = fitz.open(stream=content, filetype="pdf")
#             page_count = len(doc)
#             pdf_text = ""

#             for page_num in range(page_count):
#                 page = doc[page_num]
#                 page_text = page.get_text("text", sort=True)
#                 pdf_text += page_text + "\n"

#                 if (page_num + 1) % 10 == 0:
#                     print(f"  → Processed {page_num + 1}/{page_count} pages...")

#             doc.close()

#             # Clean the extracted text
#             pdf_text = clean_text(pdf_text)

#             all_text.append({
#                 'filename': filename,
#                 'text': pdf_text,
#                 'pages': page_count,
#                 'word_count': len(pdf_text.split())
#             })

#             print(f"✓ Extracted {page_count} pages ({len(pdf_text.split())} words)")

#         except Exception as e:
#             print(f"❌ Error processing {filename}: {e}")

#     return all_text

# def create_langchain_chunks(text, source_filename):
#     """
#     Use LangChain's RecursiveCharacterTextSplitter for intelligent chunking.
#     This splits on paragraph boundaries, then sentences, then words - preserving semantic coherence.
#     """
#     # Initialize LangChain's text splitter
#     text_splitter = RecursiveCharacterTextSplitter(
#         chunk_size=CHUNK_SIZE,
#         chunk_overlap=CHUNK_OVERLAP,
#         length_function=len,
#         separators=[
#             "\n\n",  # Split on paragraphs first
#             "\n",    # Then on line breaks
#             ". ",    # Then on sentences
#             ", ",    # Then on clauses
#             " ",     # Finally on words
#             ""       # Last resort: character level
#         ],
#         is_separator_regex=False
#     )

#     # Create chunks using LangChain
#     chunks = text_splitter.create_documents(
#         texts=[text],
#         metadatas=[{"source": source_filename}]
#     )

#     # Convert LangChain Document objects to our format
#     formatted_chunks = []
#     for idx, chunk in enumerate(chunks):
#         formatted_chunks.append({
#             'text': chunk.page_content,
#             'source': source_filename,
#             'chunk_position': idx,
#             'word_count': len(chunk.page_content.split()),
#             'char_count': len(chunk.page_content)
#         })

#     return formatted_chunks

# def call_gpt4o_for_upsc(chunk_id, text_chunk, source_file, client):
#     """Enhanced GPT-4o call with better prompt for quality crux."""
#     system_prompt = """
# You are a UPSC CSE Mains examiner creating a high-quality study database.

# STRICT ANALYSIS CRITERIA:
# 1. ONLY extract content with clear UPSC Mains relevance
# 2. Each entry must have policy/governance/constitutional/ethical/economic implications
# 3. Must be mappable to specific GS1-GS4 syllabus topics
# 4. Must have potential for 10/15-mark analytical questions

# REJECTION CRITERIA (mark as irrelevant):
# ❌ Celebrity news, entertainment, sports scores
# ❌ Simple factual updates without analytical depth
# ❌ Advertisements, promotional content
# ❌ Routine appointments without policy significance
# ❌ Local incidents without systemic lessons
# ❌ Generic news headlines without deeper implications

# ACCEPTANCE CRITERIA (mark as relevant):
# ✓ Policy announcements with governance implications
# ✓ Court judgments on constitutional matters
# ✓ Economic reforms/fiscal measures
# ✓ International relations developments
# ✓ Social issues with ethical dimensions
# ✓ Environmental/scientific developments with policy impact
# ✓ Governance case studies (success/failure)

# CRUX WRITING RULES (CRITICAL):
# The "cruz" field must follow this EXACT structure:

# **What happened:** [2-3 sentences: Clearly state the NEWS EVENT/DEVELOPMENT from the article - be specific with facts, names, dates, numbers, court names, policy names, etc.]

# **Static concept:** [1-2 sentences: Identify the UPSC syllabus topic this connects to - mention specific GS paper topic like "Article 21", "Monetary Policy", "Cooperative Federalism", etc.]

# **Why it matters for UPSC:** [2-3 sentences: Explain how this can be asked in Mains - what analytical angle, what debate it raises, what examples it provides for answers]

# **Possible question angle:** [1 sentence: Suggest a potential Mains question format]

# EXAMPLE OF GOOD CRUX:
# "**What happened:** The Supreme Court ruled that the Electoral Bond Scheme violates Article 19(1)(a) (freedom of speech) and citizens' right to information under Article 19(1)(a). The court ordered SBI to disclose all donor details to the Election Commission by March 2024.

# **Static concept:** This relates to GS2 topics: transparency in political funding, Right to Information, Article 19 fundamental rights, and election reforms.

# **Why it matters for UPSC:** This judgment is crucial for questions on electoral reforms, transparency in governance, and balancing privacy vs. public interest. It provides a recent Supreme Court precedent for RTI-related questions and can be used as a case study in answers about political finance regulation.

# **Possible question angle:** 'The Supreme Court's verdict on Electoral Bonds has reignited the debate on transparency in political funding. Critically analyze the judgment's implications for democratic accountability.'"

# RETURN FORMAT:
# Return JSON with key 'items' containing ONE object per distinct news story/theme:
# {
#   "items": [
#     {
#       "relevant": true/false,
#       "gs_paper": "GS1" or "GS2" or "GS3" or "GS4" or "Not Applicable",
#       "syllabus_topic": "Specific syllabus line" or "Not Applicable",
#       "cruz": "Follow the structure above" or "Not Applicable"
#     }
#   ]
# }

# If relevant = false:
# - gs_paper = "Not Applicable"
# - syllabus_topic = "Not Applicable"
# - cruz = "Not Applicable"

# IMPORTANT:
# - Extract ONLY distinct news stories (don't repeat same story multiple times)
# - If chunk contains multiple distinct UPSC-relevant stories, create separate items
# - If chunk is repetitive text or contains no new info, return single item with relevant=false
# """

#     try:
#         response = client.chat.completions.create(
#             model=MODEL,
#             messages=[
#                 {"role": "system", "content": system_prompt},
#                 {"role": "user", "content": f"Source: {source_file}\n\nAnalyze this newspaper chunk:\n\n{text_chunk}"}
#             ],
#             response_format={"type": "json_object"},
#             temperature=0.2
#         )
#         return json.loads(response.choices[0].message.content)
#     except Exception as e:
#         print(f"  ❌ API Error on chunk {chunk_id}: {e}")
#         return {"items": []}

# def save_extracted_text(pdf_data):
#     """Save concatenated text from all PDFs to a file for review."""
#     combined_text = ""
#     for pdf in pdf_data:
#         combined_text += f"\n\n{'='*80}\n"
#         combined_text += f"SOURCE: {pdf['filename']}\n"
#         combined_text += f"PAGES: {pdf['pages']} | WORDS: {pdf['word_count']}\n"
#         combined_text += f"{'='*80}\n\n"
#         combined_text += pdf['text']

#     with open('extracted_text_all_pdfs.txt', 'w', encoding='utf-8') as f:
#         f.write(combined_text)

#     print(f"\n✓ Saved extracted text to: extracted_text_all_pdfs.txt")
#     return 'extracted_text_all_pdfs.txt'

# # --- MAIN EXECUTION ---
# def main():
#     client = OpenAI(api_key=OPENAI_API_KEY)

#     print("="*80)
#     print(" "*15 + "UPSC NEWSPAPER ANALYSIS PIPELINE (LangChain)")
#     print("="*80)
#     print("\n📤 Upload your newspaper PDFs (multiple files supported)")
#     print("-"*80)

#     uploaded = files.upload()

#     if not uploaded:
#         print("\n❌ No files uploaded. Exiting...")
#         return

#     print(f"\n✓ {len(uploaded)} file(s) uploaded successfully\n")

#     # Extract text from all PDFs
#     pdf_data = extract_text_from_pdfs(uploaded)

#     if not pdf_data:
#         print("\n❌ No text could be extracted. Exiting...")
#         return

#     # Save extracted text for review
#     text_file = save_extracted_text(pdf_data)
#     files.download(text_file)

#     # Create chunks from all PDFs using LangChain
#     print(f"\n{'='*80}")
#     print(f"🔄 CREATING SEMANTIC CHUNKS WITH LANGCHAIN")
#     print(f"{'='*80}")
#     print(f"Chunk size: {CHUNK_SIZE} characters")
#     print(f"Overlap: {CHUNK_OVERLAP} characters")
#     print(f"{'='*80}\n")

#     all_chunks = []
#     for pdf in pdf_data:
#         if pdf['text'].strip():
#             print(f"Processing {pdf['filename']}...")
#             pdf_chunks = create_langchain_chunks(pdf['text'], pdf['filename'])
#             all_chunks.extend(pdf_chunks)

#             # Calculate average chunk size
#             avg_words = sum(c['word_count'] for c in pdf_chunks) / len(pdf_chunks) if pdf_chunks else 0
#             avg_chars = sum(c['char_count'] for c in pdf_chunks) / len(pdf_chunks) if pdf_chunks else 0

#             print(f"✓ Created {len(pdf_chunks)} semantic chunks")
#             print(f"  Average: {avg_words:.0f} words ({avg_chars:.0f} characters) per chunk\n")

#     if not all_chunks:
#         print("\n❌ No chunks created. Exiting...")
#         return

#     print(f"\n{'='*80}")
#     print(f"📊 PROCESSING SUMMARY")
#     print(f"{'='*80}")
#     print(f"Total PDFs: {len(pdf_data)}")
#     print(f"Total chunks to analyze: {len(all_chunks)}")
#     print(f"Model: {MODEL}")
#     print(f"Chunking: LangChain RecursiveCharacterTextSplitter")
#     print(f"{'='*80}\n")

#     all_data = []
#     relevant_count = 0
#     irrelevant_count = 0

#     # Process each chunk
#     for idx, chunk_obj in enumerate(all_chunks):
#         print(f"\n[{idx+1}/{len(all_chunks)}] Processing chunk from {chunk_obj['source']}...")
#         print(f"  Words: {chunk_obj['word_count']} | Characters: {chunk_obj['char_count']}")

#         # API Call
#         result = call_gpt4o_for_upsc(idx, chunk_obj['text'], chunk_obj['source'], client)

#         items = result.get("items", [])

#         if not items:
#             items = [{
#                 "relevant": False,
#                 "gs_paper": "Not Applicable",
#                 "syllabus_topic": "Not Applicable",
#                 "cruz": "Not Applicable"
#             }]

#         for item_idx, item in enumerate(items):
#             is_relevant = item.get("relevant", False)

#             all_data.append({
#                 "ID": f"UPSC_{idx:04d}_{item_idx}_{time.strftime('%Y%m%d')}",
#                 "Source PDF": chunk_obj['source'],
#                 "Text Chunk": chunk_obj['text'],
#                 "Chunk Word Count": chunk_obj['word_count'],
#                 "Chunk Char Count": chunk_obj['char_count'],
#                 "Relevant to UPSC": is_relevant,
#                 "GS Paper": item.get("gs_paper", "Not Applicable"),
#                 "Syllabus Topic": item.get("syllabus_topic", "Not Applicable"),
#                 "Crux": item.get("cruz", "Not Applicable")
#             })

#             if is_relevant:
#                 relevant_count += 1
#                 preview = chunk_obj['text'][:100] + "..." if len(chunk_obj['text']) > 100 else chunk_obj['text']
#                 print(f"  ✓ Found relevant entry: {item.get('gs_paper')} - {item.get('syllabus_topic')[:50]}...")
#                 print(f"    Preview: {preview}")
#             else:
#                 irrelevant_count += 1

#         # Rate limiting
#         time.sleep(1.5)

#     # Create DataFrame
#     print(f"\n{'='*80}")
#     print(f"✅ PROCESSING COMPLETE")
#     print(f"{'='*80}")
#     print(f"Total entries: {len(all_data)}")
#     print(f"✓ Relevant: {relevant_count}")
#     print(f"✗ Irrelevant: {irrelevant_count}")
#     print(f"Relevance rate: {(relevant_count/len(all_data)*100):.1f}%")
#     print(f"{'='*80}\n")

#     if all_data:
#         df = pd.DataFrame(all_data)

#         # Summary by PDF
#         print("📑 SUMMARY BY SOURCE:")
#         for pdf in pdf_data:
#             pdf_entries = df[df['Source PDF'] == pdf['filename']]
#             pdf_relevant = pdf_entries[pdf_entries['Relevant to UPSC'] == True]
#             print(f"  • {pdf['filename']}: {len(pdf_relevant)}/{len(pdf_entries)} relevant")

#         # Summary by GS Paper
#         print("\n📚 SUMMARY BY GS PAPER:")
#         gs_summary = df[df['Relevant to UPSC'] == True]['GS Paper'].value_counts()
#         for paper, count in gs_summary.items():
#             if paper != "Not Applicable":
#                 print(f"  • {paper}: {count} entries")

#         # Summary by Syllabus Topic (top 10)
#         print("\n📖 TOP 10 SYLLABUS TOPICS:")
#         topic_summary = df[df['Relevant to UPSC'] == True]['Syllabus Topic'].value_counts().head(10)
#         for topic, count in topic_summary.items():
#             if topic != "Not Applicable":
#                 topic_short = topic[:60] + "..." if len(topic) > 60 else topic
#                 print(f"  • {topic_short}: {count}")

#         # Save CSV
#         output_filename = f"upsc_dataset_langchain_{time.strftime('%Y%m%d_%H%M%S')}.csv"
#         df.to_csv(output_filename, index=False)

#         print(f"\n💾 Dataset saved: {output_filename}")
#         print(f"{'='*80}\n")

#         files.download(output_filename)
#     else:
#         print("\n⚠️  No data extracted.")

# if __name__ == "__main__":
#     main()

               UPSC NEWSPAPER ANALYSIS PIPELINE (LangChain)

📤 Upload your newspaper PDFs (multiple files supported)
--------------------------------------------------------------------------------


Saving Bollywood-Town-February-2026.pdf to Bollywood-Town-February-2026 (4).pdf
Saving sportstar.pdf to sportstar (3).pdf
Saving TH Delhi 04-02-2026 (1) (2).pdf to TH Delhi 04-02-2026 (1) (2) (1).pdf

✓ 3 file(s) uploaded successfully


📄 Processing: Bollywood-Town-February-2026 (4).pdf
  → Processed 10/40 pages...
  → Processed 20/40 pages...
  → Processed 30/40 pages...
  → Processed 40/40 pages...
✓ Extracted 40 pages (11577 words)

📄 Processing: sportstar (3).pdf
  → Processed 10/53 pages...
  → Processed 20/53 pages...
  → Processed 30/53 pages...
  → Processed 40/53 pages...
  → Processed 50/53 pages...
✓ Extracted 53 pages (3 words)

📄 Processing: TH Delhi 04-02-2026 (1) (2) (1).pdf
  → Processed 10/22 pages...
  → Processed 20/22 pages...
✓ Extracted 22 pages (43165 words)

✓ Saved extracted text to: extracted_text_all_pdfs.txt


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


🔄 CREATING SEMANTIC CHUNKS WITH LANGCHAIN
Chunk size: 1000 characters
Overlap: 100 characters

Processing Bollywood-Town-February-2026 (4).pdf...
✓ Created 82 semantic chunks
  Average: 147 words (891 characters) per chunk

Processing sportstar (3).pdf...
✓ Created 1 semantic chunks
  Average: 3 words (33 characters) per chunk

Processing TH Delhi 04-02-2026 (1) (2) (1).pdf...
✓ Created 306 semantic chunks
  Average: 148 words (873 characters) per chunk


📊 PROCESSING SUMMARY
Total PDFs: 3
Total chunks to analyze: 389
Model: gpt-4o
Chunking: LangChain RecursiveCharacterTextSplitter


[1/389] Processing chunk from Bollywood-Town-February-2026 (4).pdf...
  Words: 148 | Characters: 984

[2/389] Processing chunk from Bollywood-Town-February-2026 (4).pdf...
  Words: 157 | Characters: 930

[3/389] Processing chunk from Bollywood-Town-February-2026 (4).pdf...
  Words: 127 | Characters: 765

[4/389] Processing chunk from Bollywood-Town-February-2026 (4).pdf...
  Words: 113 | Characters: 916



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [9]:
# UPSC Dataset Accuracy Testing with Qwen2.5-1.5B (Student Model)
# LOCAL INFERENCE VERSION - Loads model directly in Google Colab

import pandas as pd
import json
import time
from google.colab import files
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from typing import List, Dict
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import re

# --- CONFIGURATION ---
STUDENT_MODEL = "Qwen/Qwen2.5-1.5B-Instruct"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
MAX_NEW_TOKENS = 512
TEMPERATURE = 0.7

print("="*80)
print(" "*20 + "QWEN2.5-1.5B LOCAL INFERENCE SETUP")
print("="*80)
print(f"\n🖥️  Device: {DEVICE}")
print(f"📦 Model: {STUDENT_MODEL}")

class LocalQwenEvaluator:
    """Evaluator class for testing Qwen2.5-1.5B with local inference."""

    def __init__(self, model_name: str = STUDENT_MODEL):
        self.model_name = model_name
        self.model = None
        self.tokenizer = None
        self.device = DEVICE
        self._debug_count = 0

    def load_model(self):
        """Load the model and tokenizer with comprehensive verification."""
        print(f"\n🔄 Loading {self.model_name}...")
        print("⏳ This may take a few minutes on first run...")

        try:
            print("  📝 Loading tokenizer...")
            self.tokenizer = AutoTokenizer.from_pretrained(
                self.model_name,
                trust_remote_code=True
            )

            if self.tokenizer is None:
                raise ValueError("❌ Tokenizer failed to load!")

            print(f"  ✓ Tokenizer loaded: {len(self.tokenizer)} tokens in vocabulary")
            print(f"  ✓ Special tokens: EOS={self.tokenizer.eos_token}, PAD={self.tokenizer.pad_token}")

            print("  🤖 Loading model...")
            self.model = AutoModelForCausalLM.from_pretrained(
                self.model_name,
                torch_dtype=torch.float16 if self.device == "cuda" else torch.float32,
                device_map="auto" if self.device == "cuda" else None,
                trust_remote_code=True,
                low_cpu_mem_usage=True
            )

            if self.model is None:
                raise ValueError("❌ Model failed to load!")

            if self.device == "cpu":
                self.model = self.model.to(self.device)

            self.model.eval()

            print(f"\n  🔍 Running verification checks...")

            total_params = sum(p.numel() for p in self.model.parameters())
            trainable_params = sum(p.numel() for p in self.model.parameters() if p.requires_grad)
            print(f"  ✓ Total parameters: {total_params:,} ({total_params/1e9:.2f}B)")
            print(f"  ✓ Trainable parameters: {trainable_params:,}")
            print(f"  ✓ Model in eval mode: {not self.model.training}")

            first_param_device = next(self.model.parameters()).device
            print(f"  ✓ Model device: {first_param_device}")

            first_param_dtype = next(self.model.parameters()).dtype
            print(f"  ✓ Model dtype: {first_param_dtype}")

            if self.device == "cuda":
                allocated = torch.cuda.memory_allocated() / 1024**3
                reserved = torch.cuda.memory_reserved() / 1024**3
                print(f"  ✓ GPU Memory Allocated: {allocated:.2f} GB")
                print(f"  ✓ GPU Memory Reserved: {reserved:.2f} GB")

            print(f"\n  🧪 Running test inference...")
            test_result = self._test_inference()

            if test_result:
                print(f"  ✅ Test inference SUCCESSFUL!")
                print(f"  ✓ Model is ready for evaluation")
            else:
                print(f"  ⚠️  Test inference FAILED - check configuration")

            print(f"\n✅ Model loaded and verified successfully on {self.device}!")

        except Exception as e:
            print(f"\n❌ Error loading model: {e}")
            import traceback
            traceback.print_exc()
            raise

    def _test_inference(self) -> bool:
        """Run a simple test to verify model can generate text."""
        try:
            test_messages = [
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": "Return only the word 'SUCCESS' if you understand."}
            ]

            text = self.tokenizer.apply_chat_template(
                test_messages,
                tokenize=False,
                add_generation_prompt=True
            )

            model_inputs = self.tokenizer([text], return_tensors="pt").to(self.device)

            with torch.no_grad():
                generated_ids = self.model.generate(
                    **model_inputs,
                    max_new_tokens=20,
                    do_sample=False,
                    pad_token_id=self.tokenizer.eos_token_id
                )

            generated_ids = [
                output_ids[len(input_ids):]
                for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
            ]

            response = self.tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
            print(f"  Test response: '{response[:100]}'")

            return len(response) > 0 and len(response) < 1000

        except Exception as e:
            print(f"  ❌ Test inference error: {e}")
            import traceback
            traceback.print_exc()
            return False

    def check_model_health(self):
        """Comprehensive health check of loaded model."""
        print("\n" + "="*80)
        print("🏥 MODEL HEALTH CHECK")
        print("="*80)

        checks_passed = 0
        total_checks = 7

        if self.model is not None:
            print("✓ Model object exists")
            checks_passed += 1
        else:
            print("❌ Model object is None")
            return False

        if self.tokenizer is not None:
            print("✓ Tokenizer object exists")
            checks_passed += 1
        else:
            print("❌ Tokenizer object is None")
            return False

        try:
            param_device = str(next(self.model.parameters()).device)
            if self.device in param_device or (self.device == "cuda" and "cuda" in param_device):
                print(f"✓ Model on correct device: {param_device}")
                checks_passed += 1
            else:
                print(f"❌ Model on wrong device: {param_device} (expected {self.device})")
        except Exception as e:
            print(f"❌ Cannot check device: {e}")

        if not self.model.training:
            print("✓ Model in eval mode")
            checks_passed += 1
        else:
            print("⚠️  Model in training mode (should be eval)")

        try:
            test_input = self.tokenizer("test", return_tensors="pt").to(self.device)
            print("✓ Model can accept tokenized inputs")
            checks_passed += 1
        except Exception as e:
            print(f"❌ Cannot tokenize inputs: {e}")

        try:
            with torch.no_grad():
                output = self.model.generate(**test_input, max_new_tokens=5)
            print("✓ Model can generate outputs")
            checks_passed += 1
        except Exception as e:
            print(f"❌ Cannot generate: {e}")

        if self.device == "cuda":
            try:
                free_memory = (torch.cuda.get_device_properties(0).total_memory -
                              torch.cuda.memory_allocated()) / 1024**3
                if free_memory > 0.5:
                    print(f"✓ Sufficient GPU memory: {free_memory:.2f} GB free")
                    checks_passed += 1
                else:
                    print(f"⚠️  Low GPU memory: {free_memory:.2f} GB free")
            except Exception as e:
                print(f"⚠️  Cannot check GPU memory: {e}")
        else:
            checks_passed += 1
            print("✓ CPU mode (memory check skipped)")

        print(f"\n{'='*80}")
        print(f"📊 Health Check: {checks_passed}/{total_checks} checks passed")
        print(f"{'='*80}\n")

        return checks_passed >= 6

    def call_qwen_student(self, text_chunk: str) -> Dict:
        """Call Qwen2.5-1.5B locally to classify UPSC relevance."""

        system_prompt = """You are a UPSC CSE Mains examiner. Evaluate if this newspaper content is relevant for UPSC exam preparation.

RELEVANT examples (mark as relevant=true):
- Government policies and reforms
- Court judgments on constitutional matters
- Economic policies and fiscal measures
- International relations and diplomacy
- Social issues with ethical dimensions
- Environmental policies and climate action
- Science and technology with policy impact
- Governance and administrative reforms

IRRELEVANT examples (mark as relevant=false):
- Pure entertainment news
- Sports scores without policy angle
- Celebrity gossip
- Simple crime reports
- Advertisements

Be GENEROUS in marking content as relevant if it has ANY connection to governance, policy, society, economy, international affairs, or ethics.

For RELEVANT content, identify the GS Paper:
- GS1: History, Geography, Society, Culture
- GS2: Governance, Constitution, Polity, Social Justice, International Relations
- GS3: Economy, Agriculture, Science & Tech, Environment, Security
- GS4: Ethics, Integrity, Aptitude

OUTPUT FORMAT (return ONLY this JSON, nothing else):
{
  "relevant": true,
  "gs_paper": "GS2",
  "syllabus_topic": "governance and public policy",
  "confidence": 0.85
}

OR for irrelevant:
{
  "relevant": false,
  "gs_paper": "Not Applicable",
  "syllabus_topic": "Not Applicable",
  "confidence": 0.90
}"""

        user_message = f"Evaluate this text for UPSC relevance:\n\n{text_chunk[:3500]}"

        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_message}
        ]

        try:
            text = self.tokenizer.apply_chat_template(
                messages,
                tokenize=False,
                add_generation_prompt=True
            )

            model_inputs = self.tokenizer([text], return_tensors="pt").to(self.device)

            with torch.no_grad():
                generated_ids = self.model.generate(
                    **model_inputs,
                    max_new_tokens=MAX_NEW_TOKENS,
                    temperature=TEMPERATURE,
                    do_sample=True,
                    top_p=0.9,
                    repetition_penalty=1.1,
                    pad_token_id=self.tokenizer.eos_token_id
                )

            generated_ids = [
                output_ids[len(input_ids):]
                for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
            ]

            response = self.tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

            # DEBUG: Print first 3 responses
            if self._debug_count < 3:
                print(f"\n🔍 DEBUG Response #{self._debug_count + 1}:")
                print(f"Raw output: {response[:300]}")
                self._debug_count += 1

            result = self._extract_json(response)

            if result:
                return self._validate_result(result)
            else:
                if self._debug_count <= 3:
                    print(f"  ⚠️  JSON parsing failed. Raw: {response[:200]}")
                return self._fallback_classification(response, text_chunk)

        except Exception as e:
            print(f"  ⚠️  Generation error: {str(e)[:100]}")
            return self._default_response()

    def _extract_json(self, response: str) -> Dict:
        """Extract JSON from response with multiple fallback methods."""
        # Method 1: Find JSON between curly braces (nested-aware)
        brace_count = 0
        start_idx = -1
        for i, char in enumerate(response):
            if char == '{':
                if start_idx == -1:
                    start_idx = i
                brace_count += 1
            elif char == '}':
                brace_count -= 1
                if brace_count == 0 and start_idx != -1:
                    try:
                        json_str = response[start_idx:i+1]
                        return json.loads(json_str)
                    except:
                        start_idx = -1
                        continue

        # Method 2: Remove markdown code blocks
        cleaned = re.sub(r'```json\s*|\s*```', '', response)
        json_match = re.search(r'\{[^{}]*\}', cleaned)
        if json_match:
            try:
                return json.loads(json_match.group())
            except:
                pass

        # Method 3: Simple regex
        json_match = re.search(r'\{[^{}]*\}', response)
        if json_match:
            try:
                return json.loads(json_match.group())
            except:
                pass

        # Method 4: Try entire response
        try:
            return json.loads(response.strip())
        except:
            pass

        return None

    def _fallback_classification(self, response: str, text_chunk: str) -> Dict:
        """Try to extract classification from non-JSON response."""
        response_lower = response.lower()

        relevant_keywords = ['relevant', 'upsc', 'gs1', 'gs2', 'gs3', 'gs4', 'policy', 'governance',
                            'true', 'yes', 'applicable']
        irrelevant_keywords = ['irrelevant', 'not applicable', 'entertainment', 'sports',
                              'false', 'no', 'celebrity']

        relevant_score = sum(1 for kw in relevant_keywords if kw in response_lower)
        irrelevant_score = sum(1 for kw in irrelevant_keywords if kw in response_lower)

        # Extract GS paper if mentioned
        gs_paper = "Not Applicable"
        for paper in ['GS1', 'GS2', 'GS3', 'GS4']:
            if paper.lower() in response_lower or paper in response:
                gs_paper = paper
                break

        # Make a guess based on keywords
        is_relevant = relevant_score > irrelevant_score or gs_paper != "Not Applicable"

        # If still ambiguous, check the text content itself
        if relevant_score == irrelevant_score:
            text_lower = text_chunk.lower()
            policy_keywords = ['government', 'policy', 'law', 'court', 'minister', 'parliament',
                             'bill', 'act', 'scheme', 'reform', 'supreme court', 'high court']
            if any(kw in text_lower for kw in policy_keywords):
                is_relevant = True

        return {
            "relevant": is_relevant,
            "gs_paper": gs_paper if is_relevant else "Not Applicable",
            "syllabus_topic": "Extracted from response",
            "confidence": 0.5
        }

    def _validate_result(self, result: Dict) -> Dict:
        """Validate and normalize the result dictionary."""
        validated = {
            "relevant": bool(result.get("relevant", False)),
            "gs_paper": str(result.get("gs_paper", "Not Applicable")),
            "syllabus_topic": str(result.get("syllabus_topic", "Not Applicable")),
            "confidence": float(result.get("confidence", 0.5))
        }

        if validated["relevant"]:
            if validated["gs_paper"] not in ["GS1", "GS2", "GS3", "GS4"]:
                gs_match = re.search(r'GS[1-4]', validated["gs_paper"].upper())
                if gs_match:
                    validated["gs_paper"] = gs_match.group()
                else:
                    # Default to GS2 (most common) with reduced confidence
                    validated["gs_paper"] = "GS2"
                    validated["confidence"] = max(0.3, validated["confidence"] - 0.2)
        else:
            validated["gs_paper"] = "Not Applicable"
            validated["syllabus_topic"] = "Not Applicable"

        validated["confidence"] = max(0.0, min(1.0, validated["confidence"]))

        return validated

    def _default_response(self) -> Dict:
        """Return default response on error."""
        return {
            "relevant": False,
            "gs_paper": "Not Applicable",
            "syllabus_topic": "Not Applicable",
            "confidence": 0.0
        }

def calculate_metrics(y_true: List, y_pred: List, label_name: str = "Relevance") -> Dict:
    """Calculate comprehensive evaluation metrics."""

    accuracy = accuracy_score(y_true, y_pred)
    precision, recall, f1, support = precision_recall_fscore_support(
        y_true, y_pred, average='binary', zero_division=0
    )

    cm = confusion_matrix(y_true, y_pred)

    metrics = {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1_score": f1,
        "confusion_matrix": cm,
        "support": support
    }

    print(f"\n{'='*60}")
    print(f"📊 {label_name} Metrics")
    print(f"{'='*60}")
    print(f"Accuracy:  {accuracy:.4f} ({accuracy*100:.2f}%)")
    print(f"Precision: {precision:.4f}")
    print(f"Recall:    {recall:.4f}")
    print(f"F1-Score:  {f1:.4f}")
    print(f"\nConfusion Matrix:")
    print(f"                 Predicted")
    print(f"               Neg    Pos")
    print(f"Actual Neg    {cm[0][0]:4d}   {cm[0][1]:4d}")
    print(f"       Pos    {cm[1][0]:4d}   {cm[1][1]:4d}")
    print(f"{'='*60}\n")

    return metrics

def plot_confusion_matrix(cm, title, filename):
    """Plot and save confusion matrix."""
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=['Negative', 'Positive'],
                yticklabels=['Negative', 'Positive'])
    plt.title(title)
    plt.ylabel('Actual')
    plt.xlabel('Predicted')
    plt.tight_layout()
    plt.savefig(filename, dpi=300, bbox_inches='tight')
    plt.close()
    print(f"✓ Saved confusion matrix: {filename}")

def plot_gs_paper_accuracy(df_results, filename):
    """Plot accuracy by GS Paper."""
    gs_papers = ['GS1', 'GS2', 'GS3', 'GS4']
    accuracies = []
    counts = []

    for paper in gs_papers:
        paper_df = df_results[df_results['Ground_Truth_GS_Paper'] == paper]
        if len(paper_df) > 0:
            correct = (paper_df['Ground_Truth_GS_Paper'] == paper_df['Predicted_GS_Paper']).sum()
            accuracy = correct / len(paper_df)
            accuracies.append(accuracy * 100)
            counts.append(len(paper_df))
        else:
            accuracies.append(0)
            counts.append(0)

    fig, ax1 = plt.subplots(figsize=(10, 6))

    x = np.arange(len(gs_papers))
    width = 0.35

    ax1.bar(x - width/2, accuracies, width, label='Accuracy %', color='steelblue')
    ax1.set_ylabel('Accuracy (%)', color='steelblue')
    ax1.set_xlabel('GS Paper')
    ax1.set_title('Qwen2.5-1.5B: Accuracy by GS Paper')
    ax1.set_xticks(x)
    ax1.set_xticklabels(gs_papers)
    ax1.legend(loc='upper left')
    ax1.set_ylim([0, 105])

    ax2 = ax1.twinx()
    ax2.bar(x + width/2, counts, width, label='Sample Count', color='coral', alpha=0.7)
    ax2.set_ylabel('Sample Count', color='coral')
    ax2.legend(loc='upper right')

    plt.tight_layout()
    plt.savefig(filename, dpi=300, bbox_inches='tight')
    plt.close()
    print(f"✓ Saved GS Paper accuracy plot: {filename}")

def plot_comparison_with_teacher(student_metrics, teacher_metrics_file=None):
    """Create comparison plots between student and teacher models."""

    if teacher_metrics_file:
        try:
            with open(teacher_metrics_file, 'r') as f:
                teacher_metrics = json.load(f)
        except:
            print("⚠️  Could not load teacher metrics for comparison")
            return
    else:
        print("⚠️  No teacher metrics file provided - skipping comparison")
        return

    metrics_names = ['Accuracy', 'Precision', 'Recall', 'F1-Score']
    teacher_values = [
        teacher_metrics['relevance_accuracy'],
        teacher_metrics['relevance_precision'],
        teacher_metrics['relevance_recall'],
        teacher_metrics['relevance_f1']
    ]
    student_values = [
        student_metrics['relevance_accuracy'],
        student_metrics['relevance_precision'],
        student_metrics['relevance_recall'],
        student_metrics['relevance_f1']
    ]

    x = np.arange(len(metrics_names))
    width = 0.35

    fig, ax = plt.subplots(figsize=(12, 6))
    bars1 = ax.bar(x - width/2, teacher_values, width, label='Teacher (72B)', color='#2E75B6')
    bars2 = ax.bar(x + width/2, student_values, width, label='Student (1.5B)', color='#70AD47')

    ax.set_ylabel('Score')
    ax.set_title('Teacher vs Student Model Performance Comparison')
    ax.set_xticks(x)
    ax.set_xticklabels(metrics_names)
    ax.legend()
    ax.set_ylim([0, 1.1])
    ax.grid(axis='y', alpha=0.3)

    for bars in [bars1, bars2]:
        for bar in bars:
            height = bar.get_height()
            ax.annotate(f'{height:.3f}',
                       xy=(bar.get_x() + bar.get_width() / 2, height),
                       xytext=(0, 3),
                       textcoords="offset points",
                       ha='center', va='bottom',
                       fontsize=9)

    plt.tight_layout()
    plt.savefig('teacher_vs_student_comparison.png', dpi=300, bbox_inches='tight')
    plt.close()
    print(f"✓ Saved comparison plot: teacher_vs_student_comparison.png")

    print(f"\n{'='*60}")
    print(f"📊 PERFORMANCE GAP ANALYSIS")
    print(f"{'='*60}")
    print(f"Model Size Reduction: 72B → 1.5B (97.9% smaller)")
    print(f"\nMetric Differences (Student - Teacher):")
    print(f"  Accuracy:  {student_values[0] - teacher_values[0]:+.4f} ({(student_values[0] - teacher_values[0])*100:+.2f}%)")
    print(f"  Precision: {student_values[1] - teacher_values[1]:+.4f} ({(student_values[1] - teacher_values[1])*100:+.2f}%)")
    print(f"  Recall:    {student_values[2] - teacher_values[2]:+.4f} ({(student_values[2] - teacher_values[2])*100:+.2f}%)")
    print(f"  F1-Score:  {student_values[3] - teacher_values[3]:+.4f} ({(student_values[3] - teacher_values[3])*100:+.2f}%)")

    if 'processing_time_seconds' in teacher_metrics and 'processing_time_seconds' in student_metrics:
        teacher_time = teacher_metrics['processing_time_seconds'] / teacher_metrics['total_samples']
        student_time = student_metrics['processing_time_seconds'] / student_metrics['total_samples']
        speedup = teacher_time / student_time
        print(f"\nSpeed Comparison:")
        print(f"  Teacher avg time: {teacher_time:.2f}s per sample")
        print(f"  Student avg time: {student_time:.2f}s per sample")
        print(f"  Speedup: {speedup:.2f}x faster")

    print(f"{'='*60}\n")

def test_student_model(csv_file_path: str, sample_size: int = None, teacher_metrics_file: str = None):
    """Test Qwen2.5-1.5B student model on the UPSC dataset using local inference."""

    print("\n" + "="*80)
    print(" "*20 + "QWEN2.5-1.5B STUDENT MODEL TESTING")
    print("="*80)

    print("\n📂 Loading dataset...")
    df = pd.read_csv(csv_file_path)
    print(f"✓ Loaded {len(df)} rows from {csv_file_path}")

    if sample_size and sample_size < len(df):
        df = df.sample(n=sample_size, random_state=42)
        print(f"✓ Sampled {sample_size} rows for testing")

    print(f"\n🤖 Initializing Qwen2.5-1.5B evaluator...")
    evaluator = LocalQwenEvaluator(model_name=STUDENT_MODEL)
    evaluator.load_model()

    if not evaluator.check_model_health():
        print("\n❌ Model health check FAILED! Cannot proceed with evaluation.")
        print("💡 Try restarting the runtime and loading the model again.")
        raise RuntimeError("Model health check failed")

    # Test with known relevant sample
    print("\n🧪 Testing classification with sample content...")
    test_text = """The Supreme Court ruled that the Right to Privacy is a fundamental right under Article 21 of the Constitution. This landmark judgment has significant implications for data protection laws and individual liberties in India."""

    test_result = evaluator.call_qwen_student(test_text)
    print(f"📋 Test classification result: {test_result}")
    print(f"   Expected: relevant=True, gs_paper=GS2")
    print(f"   Got: relevant={test_result['relevant']}, gs_paper={test_result['gs_paper']}")

    if test_result['relevant']:
        print("   ✅ Model correctly identified relevant content!")
    else:
        print("   ⚠️  Model marked sample as irrelevant - may be too conservative")

    print("\n✅ All systems ready - proceeding with evaluation...\n")

    predictions = []
    ground_truth_relevance = []
    predicted_relevance = []
    ground_truth_gs_paper = []
    predicted_gs_paper = []
    confidences = []

    print(f"\n🔄 Starting evaluation on {len(df)} samples...")
    print("="*80)

    start_time = time.time()

    sample_counter = 0
    for idx, row in tqdm(df.iterrows(), total=len(df), desc="Evaluating"):
        sample_counter += 1

        result = evaluator.call_qwen_student(row['Text Chunk'])

        gt_relevant = row['Relevant to UPSC']
        gt_gs_paper = row['GS Paper'] if gt_relevant else "Not Applicable"

        pred_relevant = result['relevant']
        pred_gs_paper = result['gs_paper']
        confidence = result.get('confidence', 0.0)

        predictions.append({
            'ID': row['ID'],
            'Source_PDF': row['Source PDF'],
            'Ground_Truth_Relevant': gt_relevant,
            'Predicted_Relevant': pred_relevant,
            'Ground_Truth_GS_Paper': gt_gs_paper,
            'Predicted_GS_Paper': pred_gs_paper,
            'Confidence': confidence,
            'Correct_Relevance': gt_relevant == pred_relevant,
            'Correct_GS_Paper': gt_gs_paper == pred_gs_paper if gt_relevant and pred_relevant else False
        })

        ground_truth_relevance.append(gt_relevant)
        predicted_relevance.append(pred_relevant)
        ground_truth_gs_paper.append(gt_gs_paper)
        predicted_gs_paper.append(pred_gs_paper)
        confidences.append(confidence)

        if sample_counter % 5 == 0:
            match_relevance = "✓" if gt_relevant == pred_relevant else "✗"
            match_paper = "✓" if gt_gs_paper == pred_gs_paper else "✗"
            print(f"\n[{sample_counter}/{len(df)}] ID: {row['ID']}")
            print(f"  GT: Relevant={gt_relevant}, Paper={gt_gs_paper}")
            print(f"  Pred: Relevant={pred_relevant}, Paper={pred_gs_paper}")
            print(f"  Match: Relevance {match_relevance} | GS Paper {match_paper} | Conf: {confidence:.3f}")

    elapsed_time = time.time() - start_time

    df_results = pd.DataFrame(predictions)

    print("\n" + "="*80)
    print("📈 EVALUATION RESULTS")
    print("="*80)

    relevance_metrics = calculate_metrics(
        ground_truth_relevance,
        predicted_relevance,
        "Relevance Classification"
    )

    relevant_mask = [gt and pred for gt, pred in zip(ground_truth_relevance, predicted_relevance)]
    if sum(relevant_mask) > 0:
        gs_paper_gt_filtered = [gt for gt, mask in zip(ground_truth_gs_paper, relevant_mask) if mask]
        gs_paper_pred_filtered = [pred for pred, mask in zip(predicted_gs_paper, relevant_mask) if mask]

        gs_paper_binary_gt = [1] * len(gs_paper_gt_filtered)
        gs_paper_binary_pred = [1 if gt == pred else 0
                                for gt, pred in zip(gs_paper_gt_filtered, gs_paper_pred_filtered)]

        gs_paper_accuracy = sum(gs_paper_binary_pred) / len(gs_paper_binary_pred) if len(gs_paper_binary_pred) > 0 else 0

        print(f"📊 GS Paper Classification (among relevant items)")
        print(f"{'='*60}")
        print(f"Accuracy: {gs_paper_accuracy:.4f} ({gs_paper_accuracy*100:.2f}%)")
        print(f"Samples:  {len(gs_paper_binary_pred)}")
        print(f"{'='*60}\n")

    print(f"⏱️  PERFORMANCE SUMMARY")
    print(f"{'='*60}")
    print(f"Total samples:        {len(df)}")
    print(f"Processing time:      {elapsed_time:.2f} seconds")
    print(f"Avg time per sample:  {elapsed_time/len(df):.2f} seconds")
    print(f"Avg confidence:       {np.mean(confidences):.3f}")
    print(f"{'='*60}\n")

    print(f"📚 BREAKDOWN BY GS PAPER")
    print(f"{'='*60}")
    for paper in ['GS1', 'GS2', 'GS3', 'GS4']:
        paper_df = df_results[df_results['Ground_Truth_GS_Paper'] == paper]
        if len(paper_df) > 0:
            correct = (paper_df['Ground_Truth_GS_Paper'] == paper_df['Predicted_GS_Paper']).sum()
            accuracy = correct / len(paper_df)
            print(f"{paper}: {correct}/{len(paper_df)} correct ({accuracy*100:.1f}%)")
    print(f"{'='*60}\n")

    output_csv = f"qwen1.5b_student_results_{time.strftime('%Y%m%d_%H%M%S')}.csv"
    df_results.to_csv(output_csv, index=False)
    print(f"💾 Detailed results saved: {output_csv}")

    print(f"\n📊 Generating visualizations...")
    plot_confusion_matrix(
        relevance_metrics['confusion_matrix'],
        'Qwen2.5-1.5B: Relevance Classification Confusion Matrix',
        'confusion_matrix_relevance_student.png'
    )

    plot_gs_paper_accuracy(
        df_results,
        'gs_paper_accuracy_student.png'
    )

    metrics_summary = {
        'model': STUDENT_MODEL,
        'total_samples': len(df),
        'processing_time_seconds': elapsed_time,
        'relevance_accuracy': float(relevance_metrics['accuracy']),
        'relevance_precision': float(relevance_metrics['precision']),
        'relevance_recall': float(relevance_metrics['recall']),
        'relevance_f1': float(relevance_metrics['f1_score']),
        'avg_confidence': float(np.mean(confidences)),
        'timestamp': time.strftime('%Y-%m-%d %H:%M:%S'),
        'device': DEVICE
    }

    metrics_file = f"qwen1.5b_metrics_{time.strftime('%Y%m%d_%H%M%S')}.json"
    with open(metrics_file, 'w') as f:
        json.dump(metrics_summary, f, indent=2)

    print(f"✓ Metrics summary saved: {metrics_file}")

    if teacher_metrics_file:
        plot_comparison_with_teacher(metrics_summary, teacher_metrics_file)

    print(f"\n📥 Downloading results...")
    files.download(output_csv)
    files.download(metrics_file)
    files.download('confusion_matrix_relevance_student.png')
    files.download('gs_paper_accuracy_student.png')
    if teacher_metrics_file:
        try:
            files.download('teacher_vs_student_comparison.png')
        except:
            pass

    print(f"\n{'='*80}")
    print(f"✅ TESTING COMPLETE!")
    print(f"{'='*80}")

    if DEVICE == "cuda":
        del evaluator.model
        torch.cuda.empty_cache()
        print(f"🧹 GPU memory cleared")

    return df_results, metrics_summary

# --- MAIN EXECUTION ---
if __name__ == "__main__":
    print("\n" + "="*80)
    print("⚙️  SYSTEM CHECK")
    print("="*80)
    print(f"PyTorch version: {torch.__version__}")
    print(f"CUDA available: {torch.cuda.is_available()}")
    if torch.cuda.is_available():
        print(f"CUDA version: {torch.version.cuda}")
        print(f"GPU: {torch.cuda.get_device_name(0)}")
        print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
    print("="*80)

    print("\n📤 Upload your UPSC dataset CSV file (with ground truth labels)")
    print("-"*80)

    uploaded = files.upload()

    if not uploaded:
        print("\n❌ No file uploaded. Exiting...")
    else:
        csv_file = list(uploaded.keys())[0]
        print(f"\n✓ File uploaded: {csv_file}")

        print("\n" + "="*80)
        print("⚙️  CONFIGURATION")
        print("="*80)
        print("How many samples do you want to test?")
        print("  - Enter a number (e.g., 30) for testing subset")
        print("  - Press Enter to test ALL samples")
        print("  - Recommended: Start with 30 samples to test model loading")
        print("-"*80)

        sample_input = input("Sample size: ").strip()
        sample_size = int(sample_input) if sample_input.isdigit() else None

        print("\n" + "="*80)
        print("Do you have teacher model metrics for comparison?")
        print("  - Enter the filename (e.g., qwen72b_metrics_20260215_173746.json)")
        print("  - Press Enter to skip comparison")
        print("-"*80)

        teacher_file = input("Teacher metrics file: ").strip()
        teacher_metrics_file = teacher_file if teacher_file else None

        print("\n🚀 Starting evaluation with local inference...")
        results_df, metrics = test_student_model(csv_file, sample_size, teacher_metrics_file)

        print("\n🎉 Student model evaluation complete!")
        print("💡 The 1.5B model is now running locally in your Colab environment")
        print("💡 Use these results to assess knowledge distillation effectiveness")

                    QWEN2.5-1.5B LOCAL INFERENCE SETUP

🖥️  Device: cuda
📦 Model: Qwen/Qwen2.5-1.5B-Instruct

⚙️  SYSTEM CHECK
PyTorch version: 2.9.0+cu128
CUDA available: True
CUDA version: 12.8
GPU: Tesla T4
GPU Memory: 14.56 GB

📤 Upload your UPSC dataset CSV file (with ground truth labels)
--------------------------------------------------------------------------------


Saving upsc_dataset_langchain_20260213_095213.csv to upsc_dataset_langchain_20260213_095213 (5).csv

✓ File uploaded: upsc_dataset_langchain_20260213_095213 (5).csv

⚙️  CONFIGURATION
How many samples do you want to test?
  - Enter a number (e.g., 30) for testing subset
  - Press Enter to test ALL samples
  - Recommended: Start with 30 samples to test model loading
--------------------------------------------------------------------------------
Sample size: 100

Do you have teacher model metrics for comparison?
  - Enter the filename (e.g., qwen72b_metrics_20260215_173746.json)
  - Press Enter to skip comparison
--------------------------------------------------------------------------------
Teacher metrics file: 

🚀 Starting evaluation with local inference...

                    QWEN2.5-1.5B STUDENT MODEL TESTING

📂 Loading dataset...
✓ Loaded 447 rows from upsc_dataset_langchain_20260213_095213 (5).csv
✓ Sampled 100 rows for testing

🤖 Initializing Qwen2.5-1.5B evaluator...

🔄 Loadi

Loading weights:   0%|          | 0/338 [00:00<?, ?it/s]


  🔍 Running verification checks...
  ✓ Total parameters: 1,543,714,304 (1.54B)
  ✓ Trainable parameters: 1,543,714,304
  ✓ Model in eval mode: True
  ✓ Model device: cuda:0
  ✓ Model dtype: torch.float16
  ✓ GPU Memory Allocated: 2.88 GB
  ✓ GPU Memory Reserved: 2.93 GB

  🧪 Running test inference...
  Test response: 'SUCCESS'
  ✅ Test inference SUCCESSFUL!
  ✓ Model is ready for evaluation

✅ Model loaded and verified successfully on cuda!

🏥 MODEL HEALTH CHECK
✓ Model object exists
✓ Tokenizer object exists
✓ Model on correct device: cuda:0
✓ Model in eval mode
✓ Model can accept tokenized inputs
✓ Model can generate outputs
✓ Sufficient GPU memory: 11.68 GB free

📊 Health Check: 7/7 checks passed


🧪 Testing classification with sample content...

🔍 DEBUG Response #1:
Raw output: ```json
{
  "relevant": true,
  "gs_paper": "GS2",
  "syllabus_topic": "governance and public policy",
  "confidence": 0.85
}
```
📋 Test classification result: {'relevant': True, 'gs_paper': 'GS2', 'syllabu

Evaluating:   1%|          | 1/100 [00:01<03:13,  1.95s/it]

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


✅ TESTING COMPLETE!
🧹 GPU memory cleared

🎉 Student model evaluation complete!
💡 The 1.5B model is now running locally in your Colab environment
💡 Use these results to assess knowledge distillation effectiveness


In [6]:
# UPSC Dataset Accuracy Testing with Qwen2.5-72B (Teacher Model)
# This establishes baseline accuracy before knowledge distillation

import pandas as pd
import json
import time
from google.colab import files
from google.colab import userdata
import requests
from typing import List, Dict, Tuple
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# --- CONFIGURATION ---
# You can use Hugging Face Inference API or local inference
# Option 1: Hugging Face Inference API (easiest)
HF_API_KEY = userdata.get('HF_TOKEN')  # Get your key from hf.co/settings/tokens
TEACHER_MODEL = "Qwen/Qwen2.5-72B-Instruct"

# Option 2: For local inference (uncomment if using vLLM or similar)
# LOCAL_API_URL = "http://localhost:8000/v1/chat/completions"

class QwenTeacherEvaluator:
    """Evaluator class for testing Qwen2.5-72B on UPSC dataset."""

    def __init__(self, api_key: str, model_name: str, use_hf_api: bool = True):
        self.api_key = api_key
        self.model_name = model_name
        self.use_hf_api = use_hf_api

        if use_hf_api:
            # Updated to new HF router endpoint
            self.api_url = f"https://router.huggingface.co/v1/chat/completions"
            self.headers = {
                "Authorization": f"Bearer {api_key}",
                "Content-Type": "application/json"
            }

    def call_qwen_teacher(self, text_chunk: str) -> Dict:
        """Call Qwen2.5-72B to classify UPSC relevance."""

        system_prompt = """You are a UPSC CSE Mains examiner evaluating newspaper content for exam relevance.

Your task: Analyze the given text and determine if it's relevant for UPSC preparation.

RELEVANT content includes:
- Policy announcements with governance implications
- Court judgments on constitutional matters
- Economic reforms/fiscal measures
- International relations developments
- Social issues with ethical dimensions
- Environmental/scientific developments with policy impact
- Governance case studies

IRRELEVANT content includes:
- Celebrity news, entertainment, sports
- Simple factual updates without analytical depth
- Advertisements, promotional content
- Local incidents without systemic lessons

For RELEVANT content, also identify:
1. Which GS Paper (GS1, GS2, GS3, or GS4)
2. Specific syllabus topic it maps to

Return ONLY a JSON object in this format:
{
  "relevant": true or false,
  "gs_paper": "GS1" or "GS2" or "GS3" or "GS4" or "Not Applicable",
  "syllabus_topic": "specific topic" or "Not Applicable",
  "confidence": 0.0 to 1.0
}"""

        user_message = f"Analyze this newspaper text for UPSC relevance:\n\n{text_chunk[:4000]}"  # Limit to 4k chars

        if self.use_hf_api:
            return self._call_hf_api(system_prompt, user_message)
        else:
            return self._call_local_api(system_prompt, user_message)

    def _call_hf_api(self, system_prompt: str, user_message: str) -> Dict:
        """Call Hugging Face Inference API using new router endpoint."""

        # New format uses OpenAI-compatible chat completions
        payload = {
            "model": self.model_name,
            "messages": [
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_message}
            ],
            "max_tokens": 256,
            "temperature": 0.1,
            "top_p": 0.9
        }

        max_retries = 3
        for attempt in range(max_retries):
            try:
                response = requests.post(
                    self.api_url,
                    headers=self.headers,
                    json=payload,
                    timeout=60
                )

                if response.status_code == 200:
                    result = response.json()
                    # Extract content from chat completion format
                    generated_text = result["choices"][0]["message"]["content"]

                    # Extract JSON from response
                    json_start = generated_text.find('{')
                    json_end = generated_text.rfind('}') + 1

                    if json_start != -1 and json_end > json_start:
                        json_str = generated_text[json_start:json_end]
                        return json.loads(json_str)
                    else:
                        return self._default_response()

                elif response.status_code == 503:
                    print(f"  ⏳ Model loading... retrying in 20s (attempt {attempt+1}/{max_retries})")
                    time.sleep(20)
                else:
                    print(f"  ❌ API Error {response.status_code}: {response.text}")
                    return self._default_response()

            except Exception as e:
                print(f"  ❌ Exception: {e}")
                if attempt < max_retries - 1:
                    time.sleep(5)
                else:
                    return self._default_response()

        return self._default_response()

    def _call_local_api(self, system_prompt: str, user_message: str) -> Dict:
        """Call local vLLM or similar API endpoint."""
        payload = {
            "model": self.model_name,
            "messages": [
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_message}
            ],
            "temperature": 0.1,
            "max_tokens": 256
        }

        try:
            response = requests.post(
                LOCAL_API_URL,
                json=payload,
                timeout=60
            )

            if response.status_code == 200:
                result = response.json()
                content = result["choices"][0]["message"]["content"]

                # Extract JSON
                json_start = content.find('{')
                json_end = content.rfind('}') + 1

                if json_start != -1 and json_end > json_start:
                    json_str = content[json_start:json_end]
                    return json.loads(json_str)

            return self._default_response()

        except Exception as e:
            print(f"  ❌ Local API Error: {e}")
            return self._default_response()

    def _default_response(self) -> Dict:
        """Return default response on error."""
        return {
            "relevant": False,
            "gs_paper": "Not Applicable",
            "syllabus_topic": "Not Applicable",
            "confidence": 0.0
        }

def calculate_metrics(y_true: List, y_pred: List, label_name: str = "Relevance") -> Dict:
    """Calculate comprehensive evaluation metrics."""

    accuracy = accuracy_score(y_true, y_pred)
    precision, recall, f1, support = precision_recall_fscore_support(
        y_true, y_pred, average='binary', zero_division=0
    )

    # Confusion matrix
    cm = confusion_matrix(y_true, y_pred)

    metrics = {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1_score": f1,
        "confusion_matrix": cm,
        "support": support
    }

    print(f"\n{'='*60}")
    print(f"📊 {label_name} Metrics")
    print(f"{'='*60}")
    print(f"Accuracy:  {accuracy:.4f} ({accuracy*100:.2f}%)")
    print(f"Precision: {precision:.4f}")
    print(f"Recall:    {recall:.4f}")
    print(f"F1-Score:  {f1:.4f}")
    print(f"\nConfusion Matrix:")
    print(f"                 Predicted")
    print(f"               Neg    Pos")
    print(f"Actual Neg    {cm[0][0]:4d}   {cm[0][1]:4d}")
    print(f"       Pos    {cm[1][0]:4d}   {cm[1][1]:4d}")
    print(f"{'='*60}\n")

    return metrics

def plot_confusion_matrix(cm, title, filename):
    """Plot and save confusion matrix."""
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=['Negative', 'Positive'],
                yticklabels=['Negative', 'Positive'])
    plt.title(title)
    plt.ylabel('Actual')
    plt.xlabel('Predicted')
    plt.tight_layout()
    plt.savefig(filename, dpi=300, bbox_inches='tight')
    plt.close()
    print(f"✓ Saved confusion matrix: {filename}")

def plot_gs_paper_accuracy(df_results, filename):
    """Plot accuracy by GS Paper."""
    gs_papers = ['GS1', 'GS2', 'GS3', 'GS4']
    accuracies = []
    counts = []

    for paper in gs_papers:
        paper_df = df_results[df_results['Ground_Truth_GS_Paper'] == paper]
        if len(paper_df) > 0:
            correct = (paper_df['Ground_Truth_GS_Paper'] == paper_df['Predicted_GS_Paper']).sum()
            accuracy = correct / len(paper_df)
            accuracies.append(accuracy * 100)
            counts.append(len(paper_df))
        else:
            accuracies.append(0)
            counts.append(0)

    fig, ax1 = plt.subplots(figsize=(10, 6))

    x = np.arange(len(gs_papers))
    width = 0.35

    ax1.bar(x - width/2, accuracies, width, label='Accuracy %', color='steelblue')
    ax1.set_ylabel('Accuracy (%)', color='steelblue')
    ax1.set_xlabel('GS Paper')
    ax1.set_title('Qwen2.5-72B: Accuracy by GS Paper')
    ax1.set_xticks(x)
    ax1.set_xticklabels(gs_papers)
    ax1.legend(loc='upper left')
    ax1.set_ylim([0, 105])

    ax2 = ax1.twinx()
    ax2.bar(x + width/2, counts, width, label='Sample Count', color='coral', alpha=0.7)
    ax2.set_ylabel('Sample Count', color='coral')
    ax2.legend(loc='upper right')

    plt.tight_layout()
    plt.savefig(filename, dpi=300, bbox_inches='tight')
    plt.close()
    print(f"✓ Saved GS Paper accuracy plot: {filename}")

def test_teacher_model(csv_file_path: str, sample_size: int = None):
    """
    Test Qwen2.5-72B teacher model on the UPSC dataset.

    Args:
        csv_file_path: Path to the CSV file with ground truth labels
        sample_size: Number of samples to test (None = all samples)
    """

    print("="*80)
    print(" "*20 + "QWEN2.5-72B TEACHER MODEL TESTING")
    print("="*80)

    # Load dataset
    print("\n📂 Loading dataset...")
    df = pd.read_csv(csv_file_path)
    print(f"✓ Loaded {len(df)} rows from {csv_file_path}")

    # Sample if requested
    if sample_size and sample_size < len(df):
        df = df.sample(n=sample_size, random_state=42)
        print(f"✓ Sampled {sample_size} rows for testing")

    # Initialize evaluator
    print(f"\n🤖 Initializing Qwen2.5-72B evaluator...")
    evaluator = QwenTeacherEvaluator(
        api_key=HF_API_KEY,
        model_name=TEACHER_MODEL,
        use_hf_api=True  # Change to False if using local API
    )

    # Prepare results storage
    predictions = []
    ground_truth_relevance = []
    predicted_relevance = []
    ground_truth_gs_paper = []
    predicted_gs_paper = []
    confidences = []

    print(f"\n🔄 Starting evaluation on {len(df)} samples...")
    print("="*80)

    start_time = time.time()

    # Evaluate each sample
    for idx, row in df.iterrows():
        print(f"\n[{idx+1}/{len(df)}] Processing ID: {row['ID']}")

        # Get prediction from Qwen
        result = evaluator.call_qwen_teacher(row['Text Chunk'])

        # Ground truth
        gt_relevant = row['Relevant to UPSC']
        gt_gs_paper = row['GS Paper'] if gt_relevant else "Not Applicable"

        # Prediction
        pred_relevant = result['relevant']
        pred_gs_paper = result['gs_paper']
        confidence = result.get('confidence', 0.0)

        # Store results
        predictions.append({
            'ID': row['ID'],
            'Source_PDF': row['Source PDF'],
            'Ground_Truth_Relevant': gt_relevant,
            'Predicted_Relevant': pred_relevant,
            'Ground_Truth_GS_Paper': gt_gs_paper,
            'Predicted_GS_Paper': pred_gs_paper,
            'Confidence': confidence,
            'Correct_Relevance': gt_relevant == pred_relevant,
            'Correct_GS_Paper': gt_gs_paper == pred_gs_paper if gt_relevant and pred_relevant else False
        })

        ground_truth_relevance.append(gt_relevant)
        predicted_relevance.append(pred_relevant)
        ground_truth_gs_paper.append(gt_gs_paper)
        predicted_gs_paper.append(pred_gs_paper)
        confidences.append(confidence)

        # Progress indicator
        match_relevance = "✓" if gt_relevant == pred_relevant else "✗"
        match_paper = "✓" if gt_gs_paper == pred_gs_paper else "✗"
        print(f"  Ground Truth: Relevant={gt_relevant}, Paper={gt_gs_paper}")
        print(f"  Prediction:   Relevant={pred_relevant}, Paper={pred_gs_paper}")
        print(f"  Match: Relevance {match_relevance} | GS Paper {match_paper} | Confidence: {confidence:.3f}")

        # Rate limiting for HF API
        time.sleep(1.5)

    elapsed_time = time.time() - start_time

    # Create results DataFrame
    df_results = pd.DataFrame(predictions)

    # Calculate metrics
    print("\n" + "="*80)
    print("📈 EVALUATION RESULTS")
    print("="*80)

    # Relevance classification metrics
    relevance_metrics = calculate_metrics(
        ground_truth_relevance,
        predicted_relevance,
        "Relevance Classification"
    )

    # GS Paper classification metrics (only for relevant items)
    relevant_mask = [gt and pred for gt, pred in zip(ground_truth_relevance, predicted_relevance)]
    if sum(relevant_mask) > 0:
        gs_paper_gt_filtered = [gt for gt, mask in zip(ground_truth_gs_paper, relevant_mask) if mask]
        gs_paper_pred_filtered = [pred for pred, mask in zip(predicted_gs_paper, relevant_mask) if mask]

        # Convert to binary for metrics (correct paper vs incorrect)
        gs_paper_binary_gt = [1] * len(gs_paper_gt_filtered)
        gs_paper_binary_pred = [1 if gt == pred else 0
                                for gt, pred in zip(gs_paper_gt_filtered, gs_paper_pred_filtered)]

        gs_paper_accuracy = sum(gs_paper_binary_pred) / len(gs_paper_binary_pred)

        print(f"📊 GS Paper Classification (among relevant items)")
        print(f"{'='*60}")
        print(f"Accuracy: {gs_paper_accuracy:.4f} ({gs_paper_accuracy*100:.2f}%)")
        print(f"Samples:  {len(gs_paper_binary_pred)}")
        print(f"{'='*60}\n")

    # Summary statistics
    print(f"⏱️  PERFORMANCE SUMMARY")
    print(f"{'='*60}")
    print(f"Total samples:        {len(df)}")
    print(f"Processing time:      {elapsed_time:.2f} seconds")
    print(f"Avg time per sample:  {elapsed_time/len(df):.2f} seconds")
    print(f"Avg confidence:       {np.mean(confidences):.3f}")
    print(f"{'='*60}\n")

    # Breakdown by GS Paper
    print(f"📚 BREAKDOWN BY GS PAPER")
    print(f"{'='*60}")
    for paper in ['GS1', 'GS2', 'GS3', 'GS4']:
        paper_df = df_results[df_results['Ground_Truth_GS_Paper'] == paper]
        if len(paper_df) > 0:
            correct = (paper_df['Ground_Truth_GS_Paper'] == paper_df['Predicted_GS_Paper']).sum()
            accuracy = correct / len(paper_df)
            print(f"{paper}: {correct}/{len(paper_df)} correct ({accuracy*100:.1f}%)")
    print(f"{'='*60}\n")

    # Save results
    output_csv = f"qwen72b_teacher_results_{time.strftime('%Y%m%d_%H%M%S')}.csv"
    df_results.to_csv(output_csv, index=False)
    print(f"💾 Detailed results saved: {output_csv}")

    # Generate visualizations
    print(f"\n📊 Generating visualizations...")
    plot_confusion_matrix(
        relevance_metrics['confusion_matrix'],
        'Qwen2.5-72B: Relevance Classification Confusion Matrix',
        'confusion_matrix_relevance.png'
    )

    plot_gs_paper_accuracy(
        df_results,
        'gs_paper_accuracy.png'
    )

    # Save metrics summary
    metrics_summary = {
        'model': TEACHER_MODEL,
        'total_samples': len(df),
        'processing_time_seconds': elapsed_time,
        'relevance_accuracy': float(relevance_metrics['accuracy']),
        'relevance_precision': float(relevance_metrics['precision']),
        'relevance_recall': float(relevance_metrics['recall']),
        'relevance_f1': float(relevance_metrics['f1_score']),
        'avg_confidence': float(np.mean(confidences)),
        'timestamp': time.strftime('%Y-%m-%d %H:%M:%S')
    }

    metrics_file = f"qwen72b_metrics_{time.strftime('%Y%m%d_%H%M%S')}.json"
    with open(metrics_file, 'w') as f:
        json.dump(metrics_summary, f, indent=2)

    print(f"✓ Metrics summary saved: {metrics_file}")

    # Download all files
    print(f"\n📥 Downloading results...")
    files.download(output_csv)
    files.download(metrics_file)
    files.download('confusion_matrix_relevance.png')
    files.download('gs_paper_accuracy.png')

    print(f"\n{'='*80}")
    print(f"✅ TESTING COMPLETE!")
    print(f"{'='*80}")

    return df_results, metrics_summary

# --- MAIN EXECUTION ---
if __name__ == "__main__":
    print("\n📤 Upload your UPSC dataset CSV file (with ground truth labels)")
    print("-"*80)

    uploaded = files.upload()

    if not uploaded:
        print("\n❌ No file uploaded. Exiting...")
    else:
        csv_file = list(uploaded.keys())[0]
        print(f"\n✓ File uploaded: {csv_file}")

        # Ask for sample size
        print("\n" + "="*80)
        print("⚙️  CONFIGURATION")
        print("="*80)
        print("How many samples do you want to test?")
        print("  - Enter a number (e.g., 100) for testing subset")
        print("  - Press Enter to test ALL samples")
        print("-"*80)

        sample_input = input("Sample size: ").strip()
        sample_size = int(sample_input) if sample_input.isdigit() else None

        # Run evaluation
        results_df, metrics = test_teacher_model(csv_file, sample_size)

        print("\n🎉 Teacher model baseline established!")
        print("Next step: Use these results for knowledge distillation to Qwen2.5-1.5B")


📤 Upload your UPSC dataset CSV file (with ground truth labels)
--------------------------------------------------------------------------------


Saving upsc_dataset_langchain_20260213_095213.csv to upsc_dataset_langchain_20260213_095213 (2).csv

✓ File uploaded: upsc_dataset_langchain_20260213_095213 (2).csv

⚙️  CONFIGURATION
How many samples do you want to test?
  - Enter a number (e.g., 100) for testing subset
  - Press Enter to test ALL samples
--------------------------------------------------------------------------------
Sample size: 30
                    QWEN2.5-72B TEACHER MODEL TESTING

📂 Loading dataset...
✓ Loaded 447 rows from upsc_dataset_langchain_20260213_095213 (2).csv
✓ Sampled 30 rows for testing

🤖 Initializing Qwen2.5-72B evaluator...

🔄 Starting evaluation on 30 samples...

[285/30] Processing ID: UPSC_0244_1_20260213
  Ground Truth: Relevant=False, Paper=Not Applicable
  Prediction:   Relevant=True, Paper=GS2
  Match: Relevance ✗ | GS Paper ✗ | Confidence: 0.850

[378/30] Processing ID: UPSC_0320_0_20260213


KeyboardInterrupt: 

In [7]:
# UPSC Dataset Accuracy Testing with Qwen2.5-1.5B (Student Model)
# Updated version with multiple API options

import pandas as pd
import json
import time
from google.colab import files
from google.colab import userdata
import requests
from typing import List, Dict, Tuple
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# --- CONFIGURATION ---
# Multiple API options - the script will try them in order

# Option 1: HuggingFace Inference API (Serverless)
HF_API_KEY = userdata.get('HF_TOKEN')
STUDENT_MODEL = "Qwen/Qwen2.5-1.5B-Instruct"

# Option 2: Alternative smaller models that are available
ALTERNATIVE_MODELS = [
    "Qwen/Qwen2.5-3B-Instruct",      # Slightly larger but more available
    "Qwen/Qwen2.5-7B-Instruct",      # Fallback option
    "meta-llama/Llama-3.2-1B-Instruct",  # Alternative 1B model
]

class QwenStudentEvaluator:
    """Evaluator class for testing Qwen2.5-1.5B or alternative models on UPSC dataset."""

    def __init__(self, api_key: str, model_name: str, use_serverless: bool = True):
        self.api_key = api_key
        self.model_name = model_name
        self.use_serverless = use_serverless
        self.api_type = None

        # Try different API endpoints
        self._setup_api()

    def _setup_api(self):
        """Setup API endpoint - try serverless first, then router."""

        if self.use_serverless:
            # Try HuggingFace Serverless Inference API first
            self.api_url = f"https://api-inference.huggingface.co/models/{self.model_name}"
            self.headers = {
                "Authorization": f"Bearer {self.api_key}",
                "Content-Type": "application/json"
            }
            self.api_type = "serverless"
            print(f"✓ Using HuggingFace Serverless API: {self.model_name}")
        else:
            # Router endpoint (may not support all models)
            self.api_url = f"https://router.huggingface.co/v1/chat/completions"
            self.headers = {
                "Authorization": f"Bearer {self.api_key}",
                "Content-Type": "application/json"
            }
            self.api_type = "router"
            print(f"✓ Using HuggingFace Router API: {self.model_name}")

    def call_qwen_student(self, text_chunk: str) -> Dict:
        """Call Qwen2.5-1.5B (or alternative) to classify UPSC relevance."""

        system_prompt = """You are a UPSC CSE Mains examiner evaluating newspaper content for exam relevance.

Your task: Analyze the given text and determine if it's relevant for UPSC preparation.

RELEVANT content includes:
- Policy announcements with governance implications
- Court judgments on constitutional matters
- Economic reforms/fiscal measures
- International relations developments
- Social issues with ethical dimensions
- Environmental/scientific developments with policy impact
- Governance case studies

IRRELEVANT content includes:
- Celebrity news, entertainment, sports
- Simple factual updates without analytical depth
- Advertisements, promotional content
- Local incidents without systemic lessons

For RELEVANT content, also identify:
1. Which GS Paper (GS1, GS2, GS3, or GS4)
2. Specific syllabus topic it maps to

Return ONLY a JSON object in this format:
{
  "relevant": true or false,
  "gs_paper": "GS1" or "GS2" or "GS3" or "GS4" or "Not Applicable",
  "syllabus_topic": "specific topic" or "Not Applicable",
  "confidence": 0.0 to 1.0
}"""

        user_message = f"Analyze this newspaper text for UPSC relevance:\n\n{text_chunk[:4000]}"

        if self.api_type == "serverless":
            return self._call_serverless_api(system_prompt, user_message)
        else:
            return self._call_router_api(system_prompt, user_message)

    def _call_serverless_api(self, system_prompt: str, user_message: str) -> Dict:
        """Call HuggingFace Serverless Inference API."""

        # Format for text generation models
        full_prompt = f"{system_prompt}\n\nUser: {user_message}\n\nAssistant:"

        payload = {
            "inputs": full_prompt,
            "parameters": {
                "max_new_tokens": 256,
                "temperature": 0.1,
                "top_p": 0.9,
                "return_full_text": False
            }
        }

        max_retries = 3
        for attempt in range(max_retries):
            try:
                response = requests.post(
                    self.api_url,
                    headers=self.headers,
                    json=payload,
                    timeout=60
                )

                if response.status_code == 200:
                    result = response.json()

                    # Handle different response formats
                    if isinstance(result, list) and len(result) > 0:
                        generated_text = result[0].get('generated_text', '')
                    elif isinstance(result, dict):
                        generated_text = result.get('generated_text', '')
                    else:
                        generated_text = str(result)

                    # Extract JSON from response
                    json_start = generated_text.find('{')
                    json_end = generated_text.rfind('}') + 1

                    if json_start != -1 and json_end > json_start:
                        json_str = generated_text[json_start:json_end]
                        return json.loads(json_str)
                    else:
                        return self._default_response()

                elif response.status_code == 503:
                    print(f"  ⏳ Model loading... retrying in 20s (attempt {attempt+1}/{max_retries})")
                    time.sleep(20)
                else:
                    print(f"  ❌ API Error {response.status_code}: {response.text[:200]}")
                    return self._default_response()

            except Exception as e:
                print(f"  ❌ Exception: {str(e)[:200]}")
                if attempt < max_retries - 1:
                    time.sleep(5)
                else:
                    return self._default_response()

        return self._default_response()

    def _call_router_api(self, system_prompt: str, user_message: str) -> Dict:
        """Call HuggingFace Router API (chat completions)."""

        payload = {
            "model": self.model_name,
            "messages": [
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_message}
            ],
            "max_tokens": 256,
            "temperature": 0.1,
            "top_p": 0.9
        }

        max_retries = 3
        for attempt in range(max_retries):
            try:
                response = requests.post(
                    self.api_url,
                    headers=self.headers,
                    json=payload,
                    timeout=60
                )

                if response.status_code == 200:
                    result = response.json()
                    generated_text = result["choices"][0]["message"]["content"]

                    # Extract JSON from response
                    json_start = generated_text.find('{')
                    json_end = generated_text.rfind('}') + 1

                    if json_start != -1 and json_end > json_start:
                        json_str = generated_text[json_start:json_end]
                        return json.loads(json_str)
                    else:
                        return self._default_response()

                elif response.status_code == 503:
                    print(f"  ⏳ Model loading... retrying in 20s (attempt {attempt+1}/{max_retries})")
                    time.sleep(20)
                elif response.status_code == 400:
                    print(f"  ❌ Model not supported on router API, trying serverless...")
                    # Switch to serverless and retry
                    self.api_type = "serverless"
                    self._setup_api()
                    return self._call_serverless_api(system_prompt, user_message)
                else:
                    print(f"  ❌ API Error {response.status_code}: {response.text[:200]}")
                    return self._default_response()

            except Exception as e:
                print(f"  ❌ Exception: {str(e)[:200]}")
                if attempt < max_retries - 1:
                    time.sleep(5)
                else:
                    return self._default_response()

        return self._default_response()

    def _default_response(self) -> Dict:
        """Return default response on error."""
        return {
            "relevant": False,
            "gs_paper": "Not Applicable",
            "syllabus_topic": "Not Applicable",
            "confidence": 0.0
        }

def calculate_metrics(y_true: List, y_pred: List, label_name: str = "Relevance") -> Dict:
    """Calculate comprehensive evaluation metrics."""

    accuracy = accuracy_score(y_true, y_pred)
    precision, recall, f1, support = precision_recall_fscore_support(
        y_true, y_pred, average='binary', zero_division=0
    )

    # Confusion matrix
    cm = confusion_matrix(y_true, y_pred)

    metrics = {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1_score": f1,
        "confusion_matrix": cm,
        "support": support
    }

    print(f"\n{'='*60}")
    print(f"📊 {label_name} Metrics")
    print(f"{'='*60}")
    print(f"Accuracy:  {accuracy:.4f} ({accuracy*100:.2f}%)")
    print(f"Precision: {precision:.4f}")
    print(f"Recall:    {recall:.4f}")
    print(f"F1-Score:  {f1:.4f}")
    print(f"\nConfusion Matrix:")
    print(f"                 Predicted")
    print(f"               Neg    Pos")
    print(f"Actual Neg    {cm[0][0]:4d}   {cm[0][1]:4d}")
    print(f"       Pos    {cm[1][0]:4d}   {cm[1][1]:4d}")
    print(f"{'='*60}\n")

    return metrics

def plot_confusion_matrix(cm, title, filename):
    """Plot and save confusion matrix."""
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=['Negative', 'Positive'],
                yticklabels=['Negative', 'Positive'])
    plt.title(title)
    plt.ylabel('Actual')
    plt.xlabel('Predicted')
    plt.tight_layout()
    plt.savefig(filename, dpi=300, bbox_inches='tight')
    plt.close()
    print(f"✓ Saved confusion matrix: {filename}")

def plot_gs_paper_accuracy(df_results, filename):
    """Plot accuracy by GS Paper."""
    gs_papers = ['GS1', 'GS2', 'GS3', 'GS4']
    accuracies = []
    counts = []

    for paper in gs_papers:
        paper_df = df_results[df_results['Ground_Truth_GS_Paper'] == paper]
        if len(paper_df) > 0:
            correct = (paper_df['Ground_Truth_GS_Paper'] == paper_df['Predicted_GS_Paper']).sum()
            accuracy = correct / len(paper_df)
            accuracies.append(accuracy * 100)
            counts.append(len(paper_df))
        else:
            accuracies.append(0)
            counts.append(0)

    fig, ax1 = plt.subplots(figsize=(10, 6))

    x = np.arange(len(gs_papers))
    width = 0.35

    ax1.bar(x - width/2, accuracies, width, label='Accuracy %', color='steelblue')
    ax1.set_ylabel('Accuracy (%)', color='steelblue')
    ax1.set_xlabel('GS Paper')
    ax1.set_title('Qwen2.5-1.5B: Accuracy by GS Paper')
    ax1.set_xticks(x)
    ax1.set_xticklabels(gs_papers)
    ax1.legend(loc='upper left')
    ax1.set_ylim([0, 105])

    ax2 = ax1.twinx()
    ax2.bar(x + width/2, counts, width, label='Sample Count', color='coral', alpha=0.7)
    ax2.set_ylabel('Sample Count', color='coral')
    ax2.legend(loc='upper right')

    plt.tight_layout()
    plt.savefig(filename, dpi=300, bbox_inches='tight')
    plt.close()
    print(f"✓ Saved GS Paper accuracy plot: {filename}")

def plot_comparison_with_teacher(student_metrics, teacher_metrics_file=None):
    """Create comparison plots between student and teacher models."""

    if teacher_metrics_file:
        try:
            with open(teacher_metrics_file, 'r') as f:
                teacher_metrics = json.load(f)
        except:
            print("⚠️  Could not load teacher metrics for comparison")
            return
    else:
        print("⚠️  No teacher metrics file provided - skipping comparison")
        return

    # Comparison bar chart
    metrics_names = ['Accuracy', 'Precision', 'Recall', 'F1-Score']
    teacher_values = [
        teacher_metrics['relevance_accuracy'],
        teacher_metrics['relevance_precision'],
        teacher_metrics['relevance_recall'],
        teacher_metrics['relevance_f1']
    ]
    student_values = [
        student_metrics['relevance_accuracy'],
        student_metrics['relevance_precision'],
        student_metrics['relevance_recall'],
        student_metrics['relevance_f1']
    ]

    x = np.arange(len(metrics_names))
    width = 0.35

    fig, ax = plt.subplots(figsize=(12, 6))
    bars1 = ax.bar(x - width/2, teacher_values, width, label='Teacher (72B)', color='#2E75B6')
    bars2 = ax.bar(x + width/2, student_values, width, label='Student (1.5B)', color='#70AD47')

    ax.set_ylabel('Score')
    ax.set_title('Teacher vs Student Model Performance Comparison')
    ax.set_xticks(x)
    ax.set_xticklabels(metrics_names)
    ax.legend()
    ax.set_ylim([0, 1.1])
    ax.grid(axis='y', alpha=0.3)

    # Add value labels on bars
    for bars in [bars1, bars2]:
        for bar in bars:
            height = bar.get_height()
            ax.annotate(f'{height:.3f}',
                       xy=(bar.get_x() + bar.get_width() / 2, height),
                       xytext=(0, 3),
                       textcoords="offset points",
                       ha='center', va='bottom',
                       fontsize=9)

    plt.tight_layout()
    plt.savefig('teacher_vs_student_comparison.png', dpi=300, bbox_inches='tight')
    plt.close()
    print(f"✓ Saved comparison plot: teacher_vs_student_comparison.png")

    # Performance gap analysis
    print(f"\n{'='*60}")
    print(f"📊 PERFORMANCE GAP ANALYSIS")
    print(f"{'='*60}")
    print(f"Model Size Reduction: 72B → 1.5B (97.9% smaller)")
    print(f"\nMetric Differences (Student - Teacher):")
    print(f"  Accuracy:  {student_values[0] - teacher_values[0]:+.4f} ({(student_values[0] - teacher_values[0])*100:+.2f}%)")
    print(f"  Precision: {student_values[1] - teacher_values[1]:+.4f} ({(student_values[1] - teacher_values[1])*100:+.2f}%)")
    print(f"  Recall:    {student_values[2] - teacher_values[2]:+.4f} ({(student_values[2] - teacher_values[2])*100:+.2f}%)")
    print(f"  F1-Score:  {student_values[3] - teacher_values[3]:+.4f} ({(student_values[3] - teacher_values[3])*100:+.2f}%)")

    # Speed comparison
    if 'processing_time_seconds' in teacher_metrics and 'processing_time_seconds' in student_metrics:
        teacher_time = teacher_metrics['processing_time_seconds'] / teacher_metrics['total_samples']
        student_time = student_metrics['processing_time_seconds'] / student_metrics['total_samples']
        speedup = teacher_time / student_time
        print(f"\nSpeed Comparison:")
        print(f"  Teacher avg time: {teacher_time:.2f}s per sample")
        print(f"  Student avg time: {student_time:.2f}s per sample")
        print(f"  Speedup: {speedup:.2f}x faster")

    print(f"{'='*60}\n")

def test_student_model(csv_file_path: str, sample_size: int = None, teacher_metrics_file: str = None, model_name: str = STUDENT_MODEL):
    """
    Test Qwen2.5-1.5B student model on the UPSC dataset.

    Args:
        csv_file_path: Path to the CSV file with ground truth labels
        sample_size: Number of samples to test (None = all samples)
        teacher_metrics_file: Path to teacher model metrics JSON for comparison
        model_name: Model to use (default: Qwen/Qwen2.5-1.5B-Instruct)
    """

    print("="*80)
    print(" "*20 + "QWEN2.5-1.5B STUDENT MODEL TESTING")
    print("="*80)

    # Load dataset
    print("\n📂 Loading dataset...")
    df = pd.read_csv(csv_file_path)
    print(f"✓ Loaded {len(df)} rows from {csv_file_path}")

    # Sample if requested
    if sample_size and sample_size < len(df):
        df = df.sample(n=sample_size, random_state=42)
        print(f"✓ Sampled {sample_size} rows for testing")

    # Initialize evaluator - try serverless first
    print(f"\n🤖 Initializing {model_name} evaluator...")
    evaluator = QwenStudentEvaluator(
        api_key=HF_API_KEY,
        model_name=model_name,
        use_serverless=True  # Start with serverless API
    )

    # Prepare results storage
    predictions = []
    ground_truth_relevance = []
    predicted_relevance = []
    ground_truth_gs_paper = []
    predicted_gs_paper = []
    confidences = []

    print(f"\n🔄 Starting evaluation on {len(df)} samples...")
    print("="*80)

    start_time = time.time()

    # Evaluate each sample
    for idx, row in df.iterrows():
        print(f"\n[{idx+1}/{len(df)}] Processing ID: {row['ID']}")

        # Get prediction from model
        result = evaluator.call_qwen_student(row['Text Chunk'])

        # Ground truth
        gt_relevant = row['Relevant to UPSC']
        gt_gs_paper = row['GS Paper'] if gt_relevant else "Not Applicable"

        # Prediction
        pred_relevant = result['relevant']
        pred_gs_paper = result['gs_paper']
        confidence = result.get('confidence', 0.0)

        # Store results
        predictions.append({
            'ID': row['ID'],
            'Source_PDF': row['Source PDF'],
            'Ground_Truth_Relevant': gt_relevant,
            'Predicted_Relevant': pred_relevant,
            'Ground_Truth_GS_Paper': gt_gs_paper,
            'Predicted_GS_Paper': pred_gs_paper,
            'Confidence': confidence,
            'Correct_Relevance': gt_relevant == pred_relevant,
            'Correct_GS_Paper': gt_gs_paper == pred_gs_paper if gt_relevant and pred_relevant else False
        })

        ground_truth_relevance.append(gt_relevant)
        predicted_relevance.append(pred_relevant)
        ground_truth_gs_paper.append(gt_gs_paper)
        predicted_gs_paper.append(pred_gs_paper)
        confidences.append(confidence)

        # Progress indicator
        match_relevance = "✓" if gt_relevant == pred_relevant else "✗"
        match_paper = "✓" if gt_gs_paper == pred_gs_paper else "✗"
        print(f"  Ground Truth: Relevant={gt_relevant}, Paper={gt_gs_paper}")
        print(f"  Prediction:   Relevant={pred_relevant}, Paper={pred_gs_paper}")
        print(f"  Match: Relevance {match_relevance} | GS Paper {match_paper} | Confidence: {confidence:.3f}")

        # Rate limiting - serverless API needs more time
        time.sleep(2.0)

    elapsed_time = time.time() - start_time

    # Create results DataFrame
    df_results = pd.DataFrame(predictions)

    # Calculate metrics
    print("\n" + "="*80)
    print("📈 EVALUATION RESULTS")
    print("="*80)

    # Relevance classification metrics
    relevance_metrics = calculate_metrics(
        ground_truth_relevance,
        predicted_relevance,
        "Relevance Classification"
    )

    # GS Paper classification metrics (only for relevant items)
    relevant_mask = [gt and pred for gt, pred in zip(ground_truth_relevance, predicted_relevance)]
    if sum(relevant_mask) > 0:
        gs_paper_gt_filtered = [gt for gt, mask in zip(ground_truth_gs_paper, relevant_mask) if mask]
        gs_paper_pred_filtered = [pred for pred, mask in zip(predicted_gs_paper, relevant_mask) if mask]

        # Convert to binary for metrics (correct paper vs incorrect)
        gs_paper_binary_gt = [1] * len(gs_paper_gt_filtered)
        gs_paper_binary_pred = [1 if gt == pred else 0
                                for gt, pred in zip(gs_paper_gt_filtered, gs_paper_pred_filtered)]

        gs_paper_accuracy = sum(gs_paper_binary_pred) / len(gs_paper_binary_pred) if len(gs_paper_binary_pred) > 0 else 0

        print(f"📊 GS Paper Classification (among relevant items)")
        print(f"{'='*60}")
        print(f"Accuracy: {gs_paper_accuracy:.4f} ({gs_paper_accuracy*100:.2f}%)")
        print(f"Samples:  {len(gs_paper_binary_pred)}")
        print(f"{'='*60}\n")

    # Summary statistics
    print(f"⏱️  PERFORMANCE SUMMARY")
    print(f"{'='*60}")
    print(f"Total samples:        {len(df)}")
    print(f"Processing time:      {elapsed_time:.2f} seconds")
    print(f"Avg time per sample:  {elapsed_time/len(df):.2f} seconds")
    print(f"Avg confidence:       {np.mean(confidences):.3f}")
    print(f"{'='*60}\n")

    # Breakdown by GS Paper
    print(f"📚 BREAKDOWN BY GS PAPER")
    print(f"{'='*60}")
    for paper in ['GS1', 'GS2', 'GS3', 'GS4']:
        paper_df = df_results[df_results['Ground_Truth_GS_Paper'] == paper]
        if len(paper_df) > 0:
            correct = (paper_df['Ground_Truth_GS_Paper'] == paper_df['Predicted_GS_Paper']).sum()
            accuracy = correct / len(paper_df)
            print(f"{paper}: {correct}/{len(paper_df)} correct ({accuracy*100:.1f}%)")
    print(f"{'='*60}\n")

    # Save results
    output_csv = f"qwen1.5b_student_results_{time.strftime('%Y%m%d_%H%M%S')}.csv"
    df_results.to_csv(output_csv, index=False)
    print(f"💾 Detailed results saved: {output_csv}")

    # Generate visualizations
    print(f"\n📊 Generating visualizations...")
    plot_confusion_matrix(
        relevance_metrics['confusion_matrix'],
        'Qwen2.5-1.5B: Relevance Classification Confusion Matrix',
        'confusion_matrix_relevance_student.png'
    )

    plot_gs_paper_accuracy(
        df_results,
        'gs_paper_accuracy_student.png'
    )

    # Save metrics summary
    metrics_summary = {
        'model': model_name,
        'total_samples': len(df),
        'processing_time_seconds': elapsed_time,
        'relevance_accuracy': float(relevance_metrics['accuracy']),
        'relevance_precision': float(relevance_metrics['precision']),
        'relevance_recall': float(relevance_metrics['recall']),
        'relevance_f1': float(relevance_metrics['f1_score']),
        'avg_confidence': float(np.mean(confidences)),
        'timestamp': time.strftime('%Y-%m-%d %H:%M:%S')
    }

    metrics_file = f"qwen1.5b_metrics_{time.strftime('%Y%m%d_%H%M%S')}.json"
    with open(metrics_file, 'w') as f:
        json.dump(metrics_summary, f, indent=2)

    print(f"✓ Metrics summary saved: {metrics_file}")

    # Generate comparison with teacher model if available
    if teacher_metrics_file:
        plot_comparison_with_teacher(metrics_summary, teacher_metrics_file)

    # Download all files
    print(f"\n📥 Downloading results...")
    files.download(output_csv)
    files.download(metrics_file)
    files.download('confusion_matrix_relevance_student.png')
    files.download('gs_paper_accuracy_student.png')
    if teacher_metrics_file:
        try:
            files.download('teacher_vs_student_comparison.png')
        except:
            pass

    print(f"\n{'='*80}")
    print(f"✅ TESTING COMPLETE!")
    print(f"{'='*80}")

    return df_results, metrics_summary

# --- MAIN EXECUTION ---
if __name__ == "__main__":
    print("\n📤 Upload your UPSC dataset CSV file (with ground truth labels)")
    print("-"*80)

    uploaded = files.upload()

    if not uploaded:
        print("\n❌ No file uploaded. Exiting...")
    else:
        csv_file = list(uploaded.keys())[0]
        print(f"\n✓ File uploaded: {csv_file}")

        # Ask for sample size
        print("\n" + "="*80)
        print("⚙️  CONFIGURATION")
        print("="*80)
        print("How many samples do you want to test?")
        print("  - Enter a number (e.g., 100) for testing subset")
        print("  - Press Enter to test ALL samples")
        print("-"*80)

        sample_input = input("Sample size: ").strip()
        sample_size = int(sample_input) if sample_input.isdigit() else None

        # Ask for teacher metrics file for comparison
        print("\n" + "="*80)
        print("Do you have teacher model metrics for comparison?")
        print("  - Enter the filename (e.g., qwen72b_metrics_20260215_173746.json)")
        print("  - Press Enter to skip comparison")
        print("-"*80)

        teacher_file = input("Teacher metrics file: ").strip()
        teacher_metrics_file = teacher_file if teacher_file else None

        # Model selection
        print("\n" + "="*80)
        print("⚙️  MODEL SELECTION")
        print("="*80)
        print("Select model to test:")
        print("  1. Qwen/Qwen2.5-1.5B-Instruct (default)")
        print("  2. Qwen/Qwen2.5-3B-Instruct (more available)")
        print("  3. meta-llama/Llama-3.2-1B-Instruct (alternative)")
        print("  Press Enter for default (1.5B)")
        print("-"*80)

        model_choice = input("Model choice (1-3): ").strip()

        if model_choice == "2":
            selected_model = "Qwen/Qwen2.5-3B-Instruct"
        elif model_choice == "3":
            selected_model = "meta-llama/Llama-3.2-1B-Instruct"
        else:
            selected_model = STUDENT_MODEL

        print(f"\n✓ Selected model: {selected_model}")

        # Run evaluation
        results_df, metrics = test_student_model(csv_file, sample_size, teacher_metrics_file, selected_model)

        print("\n🎉 Student model evaluation complete!")
        print("💡 Use these results to assess knowledge distillation effectiveness")


📤 Upload your UPSC dataset CSV file (with ground truth labels)
--------------------------------------------------------------------------------


Saving upsc_dataset_langchain_20260213_095213.csv to upsc_dataset_langchain_20260213_095213 (4).csv

✓ File uploaded: upsc_dataset_langchain_20260213_095213 (4).csv

⚙️  CONFIGURATION
How many samples do you want to test?
  - Enter a number (e.g., 100) for testing subset
  - Press Enter to test ALL samples
--------------------------------------------------------------------------------
Sample size: 30

Do you have teacher model metrics for comparison?
  - Enter the filename (e.g., qwen72b_metrics_20260215_173746.json)
  - Press Enter to skip comparison
--------------------------------------------------------------------------------
Teacher metrics file: 

⚙️  MODEL SELECTION
Select model to test:
  1. Qwen/Qwen2.5-1.5B-Instruct (default)
  2. Qwen/Qwen2.5-3B-Instruct (more available)
  3. meta-llama/Llama-3.2-1B-Instruct (alternative)
  Press Enter for default (1.5B)
--------------------------------------------------------------------------------
Model choice (1-3): 3

✓ Selected model

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


✅ TESTING COMPLETE!

🎉 Student model evaluation complete!
💡 Use these results to assess knowledge distillation effectiveness


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import AutoModelForCausalLM, AutoTokenizer, AdamW
from torch.utils.data import DataLoader, Dataset
import pandas as pd

# --- CONFIGURATION ---
TEACHER_ID = "Qwen/Qwen2.5-72B-Instruct" # Or a quantized version for Colab
STUDENT_ID = "Qwen/Qwen2.5-1.5B-Instruct"
TEMPERATURE = 2.0  # Softens the logit distribution
ALPHA = 0.5        # Weighting between Distillation and Student loss
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

class DistillationTrainer:
    def __init__(self, teacher_model, student_model):
        self.teacher = teacher_model.to(DEVICE).eval()
        self.student = student_model.to(DEVICE).train()

    def distillation_loss(self, student_logits, teacher_logits, labels):
        # 1. Soft Targets (KL Divergence)
        # We divide by T (Temperature) to smooth the probability distribution
        soft_targets = F.softmax(teacher_logits / TEMPERATURE, dim=-1)
        soft_prob = F.log_softmax(student_logits / TEMPERATURE, dim=-1)

        # KL Divergence loss multiplied by T^2 as per original Hinton paper
        distill_loss = F.kl_div(soft_prob, soft_targets, reduction='batchmean') * (TEMPERATURE**2)

        # 2. Hard Targets (Standard Cross-Entropy)
        student_loss = F.cross_entropy(student_logits.view(-1, student_logits.size(-1)), labels.view(-1))

        # Weighted Total Loss
        return (ALPHA * distill_loss) + ((1 - ALPHA) * student_loss)

# --- DATA PREPARATION ---
class GoldenDataset(Dataset):
    def __init__(self, csv_file, tokenizer, max_length=512):
        self.df = pd.read_csv(csv_file)
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self): return len(self.df)

    def __getitem__(self, idx):
        text = self.df.iloc[idx]['Text Chunk']
        inputs = self.tokenizer(text, truncation=True, padding='max_length',
                                max_length=self.max_length, return_tensors="pt")
        return inputs.input_ids.squeeze(), inputs.attention_mask.squeeze()

# --- TRAINING LOOP ---
def train_step(batch, trainer, optimizer):
    input_ids, attention_mask = [b.to(DEVICE) for b in batch]

    # Get Teacher Logits (No Gradient)
    with torch.no_grad():
        teacher_outputs = trainer.teacher(input_ids, attention_mask=attention_mask)
        teacher_logits = teacher_outputs.logits

    # Get Student Logits
    student_outputs = trainer.student(input_ids, attention_mask=attention_mask)
    student_logits = student_outputs.logits

    # Calculate Distillation Loss
    loss = trainer.distillation_loss(student_logits, teacher_logits, input_ids)

    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    return loss.item()

# Usage Example:
# tokenizer = AutoTokenizer.from_pretrained(STUDENT_ID)
# student_model = AutoModelForCausalLM.from_pretrained(STUDENT_ID)
# teacher_model = AutoModelForCausalLM.from_pretrained(TEACHER_ID, load_in_4bit=True)
# trainer = DistillationTrainer(teacher_model, student_model)